In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

In [2]:
stemmer=LancasterStemmer()

In [3]:
import numpy as np
import tensorflow as tf
import random
import json
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
with open("intents.json",encoding='utf8') as file:
    data=json.load(file)

In [5]:
print(data)

{'intents': [{'tag': 'coronavirus', 'patterns': ['What is a coronavirus?', 'coronavirus', 'corona'], 'responses': ['Coronaviruses are a large family of viruses which may cause illness in animals or humans.', 'In humans, several  coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases'], 'context_set': ''}, {'tag': 'COVID-19', 'patterns': ['What is COVID-19?', 'COVID-19', 'covid-19'], 'responses': ['COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019.'], 'context_set': ''}, {'tag': 'COVID-19 symptoms', 'patterns': ['What are the symptoms of COVID-19?', 'symptoms'], 'responses': ['The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea. These symptoms are usually mild and begin gradually.', "S

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
words=[]
labels=[]
docs_x=[]
docs_y=[]

In [8]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(pattern)
        docs_y.append(intent["tag"])
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
        
words=[stemmer.stem(w.lower()) for w in words if w not in "?"]
words=sorted(list(set(words)))

labels=sorted(labels)

In [9]:
enc=LabelEncoder()
enc.fit(docs_y)
docs_y=enc.transform(docs_y)

In [10]:
vocab_size=10000
embedding_dim=16
max_len=20
trunc_type='post'
oov_token="<OOV>"

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_token)#adding out of vocabulary token
tokenizer.fit_on_texts(docs_x)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(docs_x)
padded=pad_sequences(sequences,truncating=trunc_type,maxlen=max_len)

In [11]:
classes=len(labels)

In [12]:
model=keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation='relu'))
model.add(keras.layers.Dense(16,activation='relu'))
model.add(keras.layers.Dense(classes,activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
Total params: 161,088
Trainable params: 161,088
Non-trainable params: 0
_________________________________________________________________


In [14]:
training_labels_final=np.array(docs_y)

In [15]:
EPOCHS=500
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history=model.fit(padded,training_labels_final,epochs=EPOCHS)

Epoch 1/500
2/2 [==============================] - 0s 0s/step - loss: 3.4654 - accuracy: 0.0161
Epoch 2/500
2/2 [==============================] - 0s 1ms/step - loss: 3.4638 - accuracy: 0.1129
Epoch 3/500
2/2 [==============================] - 0s 1ms/step - loss: 3.4621 - accuracy: 0.1129
Epoch 4/500
2/2 [==============================] - 0s 2ms/step - loss: 3.4606 - accuracy: 0.1129
Epoch 5/500
2/2 [==============================] - 0s 1ms/step - loss: 3.4592 - accuracy: 0.1129
Epoch 6/500
2/2 [==============================] - 0s 1ms/step - loss: 3.4574 - accuracy: 0.1129
Epoch 7/500
2/2 [==============================] - 0s 997us/step - loss: 3.4559 - accuracy: 0.1129
Epoch 8/500
2/2 [==============================] - 0s 2ms/step - loss: 3.4540 - accuracy: 0.1129
Epoch 9/500
2/2 [==============================] - 0s 1ms/step - loss: 3.4521 - accuracy: 0.1129
Epoch 10/500
2/2 [==============================] - 0s 1ms/step - loss: 3.4501 - accuracy: 0.1613
Epoch 11/500
2/2 [==========

2/2 [==============================] - 0s 1ms/step - loss: 2.6600 - accuracy: 0.2258
Epoch 85/500
2/2 [==============================] - 0s 997us/step - loss: 2.6488 - accuracy: 0.2258
Epoch 86/500
2/2 [==============================] - 0s 1ms/step - loss: 2.6369 - accuracy: 0.2258
Epoch 87/500
2/2 [==============================] - 0s 1ms/step - loss: 2.6261 - accuracy: 0.2258
Epoch 88/500
2/2 [==============================] - 0s 1ms/step - loss: 2.6142 - accuracy: 0.2419
Epoch 89/500
2/2 [==============================] - 0s 1ms/step - loss: 2.6029 - accuracy: 0.2419
Epoch 90/500
2/2 [==============================] - 0s 1ms/step - loss: 2.5913 - accuracy: 0.2419
Epoch 91/500
2/2 [==============================] - 0s 1ms/step - loss: 2.5802 - accuracy: 0.2419
Epoch 92/500
2/2 [==============================] - 0s 997us/step - loss: 2.5686 - accuracy: 0.2419
Epoch 93/500
2/2 [==============================] - 0s 997us/step - loss: 2.5579 - accuracy: 0.2419
Epoch 94/500
2/2 [=========

2/2 [==============================] - 0s 1ms/step - loss: 1.7195 - accuracy: 0.3710
Epoch 167/500
2/2 [==============================] - 0s 997us/step - loss: 1.7097 - accuracy: 0.3710
Epoch 168/500
2/2 [==============================] - 0s 991us/step - loss: 1.6988 - accuracy: 0.3710
Epoch 169/500
2/2 [==============================] - 0s 1ms/step - loss: 1.6876 - accuracy: 0.3871
Epoch 170/500
2/2 [==============================] - 0s 1ms/step - loss: 1.6773 - accuracy: 0.3871
Epoch 171/500
2/2 [==============================] - 0s 1ms/step - loss: 1.6676 - accuracy: 0.3871
Epoch 172/500
2/2 [==============================] - 0s 1ms/step - loss: 1.6560 - accuracy: 0.3871
Epoch 173/500
2/2 [==============================] - 0s 1ms/step - loss: 1.6454 - accuracy: 0.3871
Epoch 174/500
2/2 [==============================] - 0s 1ms/step - loss: 1.6348 - accuracy: 0.3871
Epoch 175/500
2/2 [==============================] - 0s 1ms/step - loss: 1.6249 - accuracy: 0.3871
Epoch 176/500
2/2 [=

2/2 [==============================] - 0s 1ms/step - loss: 0.9137 - accuracy: 0.8387
Epoch 249/500
2/2 [==============================] - 0s 1ms/step - loss: 0.9055 - accuracy: 0.8387
Epoch 250/500
2/2 [==============================] - 0s 997us/step - loss: 0.8962 - accuracy: 0.8387
Epoch 251/500
2/2 [==============================] - 0s 998us/step - loss: 0.8886 - accuracy: 0.8226
Epoch 252/500
2/2 [==============================] - 0s 2ms/step - loss: 0.8797 - accuracy: 0.8226
Epoch 253/500
2/2 [==============================] - 0s 997us/step - loss: 0.8703 - accuracy: 0.8387
Epoch 254/500
2/2 [==============================] - 0s 1ms/step - loss: 0.8622 - accuracy: 0.8710
Epoch 255/500
2/2 [==============================] - 0s 1ms/step - loss: 0.8539 - accuracy: 0.8871
Epoch 256/500
2/2 [==============================] - 0s 1ms/step - loss: 0.8465 - accuracy: 0.8871
Epoch 257/500
2/2 [==============================] - 0s 1ms/step - loss: 0.8374 - accuracy: 0.8871
Epoch 258/500
2/2 

Epoch 330/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3754 - accuracy: 0.9839
Epoch 331/500
2/2 [==============================] - 0s 997us/step - loss: 0.3715 - accuracy: 0.9839
Epoch 332/500
2/2 [==============================] - 0s 997us/step - loss: 0.3673 - accuracy: 0.9839
Epoch 333/500
2/2 [==============================] - 0s 2ms/step - loss: 0.3620 - accuracy: 0.9839
Epoch 334/500
2/2 [==============================] - 0s 998us/step - loss: 0.3595 - accuracy: 0.9839
Epoch 335/500
2/2 [==============================] - 0s 993us/step - loss: 0.3539 - accuracy: 0.9839
Epoch 336/500
2/2 [==============================] - 0s 998us/step - loss: 0.3495 - accuracy: 0.9839
Epoch 337/500
2/2 [==============================] - 0s 997us/step - loss: 0.3452 - accuracy: 0.9839
Epoch 338/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3413 - accuracy: 0.9839
Epoch 339/500
2/2 [==============================] - 0s 997us/step - loss: 0.3366 - accuracy: 0.9

2/2 [==============================] - 0s 1ms/step - loss: 0.1479 - accuracy: 0.9839
Epoch 413/500
2/2 [==============================] - 0s 1ms/step - loss: 0.1468 - accuracy: 0.9839
Epoch 414/500
2/2 [==============================] - 0s 997us/step - loss: 0.1454 - accuracy: 0.9839
Epoch 415/500
2/2 [==============================] - 0s 1ms/step - loss: 0.1437 - accuracy: 0.9839
Epoch 416/500
2/2 [==============================] - 0s 1ms/step - loss: 0.1424 - accuracy: 0.9839
Epoch 417/500
2/2 [==============================] - 0s 997us/step - loss: 0.1408 - accuracy: 0.9839
Epoch 418/500
2/2 [==============================] - 0s 1ms/step - loss: 0.1393 - accuracy: 1.0000
Epoch 419/500
2/2 [==============================] - 0s 1ms/step - loss: 0.1381 - accuracy: 1.0000
Epoch 420/500
2/2 [==============================] - 0s 1ms/step - loss: 0.1368 - accuracy: 0.9839
Epoch 421/500
2/2 [==============================] - 0s 1ms/step - loss: 0.1354 - accuracy: 1.0000
Epoch 422/500
2/2 [=

2/2 [==============================] - 0s 1ms/step - loss: 0.0700 - accuracy: 1.0000
Epoch 495/500
2/2 [==============================] - 0s 2ms/step - loss: 0.0695 - accuracy: 1.0000
Epoch 496/500
2/2 [==============================] - 0s 998us/step - loss: 0.0688 - accuracy: 1.0000
Epoch 497/500
2/2 [==============================] - 0s 998us/step - loss: 0.0683 - accuracy: 1.0000
Epoch 498/500
2/2 [==============================] - 0s 1ms/step - loss: 0.0677 - accuracy: 1.0000
Epoch 499/500
2/2 [==============================] - 0s 2ms/step - loss: 0.0673 - accuracy: 1.0000
Epoch 500/500
2/2 [==============================] - 0s 1ms/step - loss: 0.0667 - accuracy: 1.0000


In [16]:
def check():
    print("Start talking to the chatbot ,Enter quit to exit")
    while True:
        string=input('Enter: ')
        if string=='quit':
            break
        result=model.predict(pad_sequences(tokenizer.texts_to_sequences([string]),
                             truncating=trunc_type,maxlen=max_len))
        category=enc.inverse_transform([np.argmax(result)])#argmax=which ever argument from our json data has maximum probabilty will be passed,inverse_transform means we are converting it back from the label encoded version 
        for i in data['intents']:
            if i in data['intents']:
                if i['tag']==category:
                    print(np.random.choice(i['responses']))#np.random.choice means it will randomly choose an response eg. if we enter hi 3 or 4 times then after saome time it will give diff response to same input

In [17]:
check()

Start talking to the chatbot ,Enter quit to exit
Enter: corona
Coronaviruses are a large family of viruses which may cause illness in animals or humans.
Enter: quit


In [18]:
tf.keras.models.save_model(model,'v_bot')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: v_bot\assets
